# Previsão do preço de vendas de tratores de esteira utilizando Machine Learning // Predicting de Sale Price of Bulldozers using Machine Leraning

Este notebook tem o propósito de elaborar um modelo de regressão para prever o preço de um trator de esteira. // This notebook attempts to through an example machine leraning project with the goal of predicting the sale price of bulldozers.

# 1. Definição do problema // Problem definition

O quão preciso conseguimos prever o preço de venda de tratatores de esteira no futuro, dadas as suas características e preços de tratos vendedidos anteriormentente? //  How well can we predict the future sale price of a bulldozer, given it's characteristics and previous examples of how much similar bulldozers have been sold for?

# 2. Dados // Data

A base de dados foi obtida no Kaggle: https://www.kaggle.com/c/bluebook-for-bulldozers/. Esta base consiste em 3
datasets:

- Train.csv é a base de dados separada para treinar os modelos e que consiste em dados do final de 2011.
- Valid.csv é a base de validação dos modelos e que consiste em dados de 01 de Janeiro de 2012 a 30 de Abril de 2012.
- Test.csv é a base de dados para testes,que contém dados de 01 de Maio de 2012 a Novembro de 2012.

//

The dataset is downloaded from Kaggle: https://www.kaggle.com/c/bluebook-for-bulldozers/. The data is split into three parts:

- Train.csv is the training set, which contains data through the end of 2011.
- Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012.
- Test.csv is the test set and contains data from May 1, 2012 - November 2012.

# 3. Avaliação do modelo // Evalutuations

A métrica de avaliação do modelo será o erro logarítmico médio quadrático da raiz entre o preço calculado através do modelo e o preço real. // The evaluation metric is the RMSLE (root mean squared log error) between the actual and predicted auction prices.
# 4. Atributos // Features
# 5. 
#
# 4. Atributos // Features
# 5. 
#